Chengyu Dai, Cupjin Huang, Kevin Sung

generated by Jupyter notebook

In [156]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import networkx as nx
from networkx.algorithms import bipartite

import itertools

# Problem Statement:
In the generalized assignment problem, we are given a collection of $n$ jobs to be assigned to $m$ machines. Each job $j = 1, . . . , n$ is to be assigned to exactly one machine; if it is assigned to machine $i$, then it requires $p_{ij}$ time units of processing, and incurs a cost of $c_{ij}$. Furthermore, we are given a time bound $T$ that limits the total processing of each machine $i = 1, . . . , m$. The aim is to find a feasible assignment of minimum total cost.


- Input : 

 + number of jobs $n$,
 
 + number of machines $m$,
 
 + cost matrix $[c_{ij}]$,
 
 + processing time matrix $[p_{ij}]$,
 
 + time bound $T$.
 
- Output: $w\in [m]^n$, where $w_i$ is the machine where job $i$ is assigned to

## IP Formulation:
Denote $x_{ij}$ be the indicator of whether job $i$ is assigned to machine $j$. Then our problem can be formulated as the following IP:

\begin{equation*}
\begin{array}{ll@{}ll}
\text{minimize}  & \displaystyle\sum\limits_{i,j} x_{ij}&c_{ij} &\\
\text{subject to}& \displaystyle\sum\limits_{j=1}^m   &x_{ij} = 1,  &i=1 ,..., n\\
                  & \displaystyle\sum\limits_{i=1}^n   &p_{ij}x_{ij} \leq  T,  &j=1 ,\cdots, m\\
                 &                                                &x_{ij} \in \{0,1\}, &i=1,\cdots,n,\\
                 & & &j=1 ,\cdots, m
\end{array}
\end{equation*}

# Approximation Algorihm
The IP above cannot be solved both exactly and efficiently assuming $\mathcal{NP}\neq \mathcal{P}$. Here we will present an algorithm which gives a solution such that

- The cost is at most the optimal cost of the original problem;

- Each machine terminates before time $2T$.

The algorithm consists of three parts:

1. Solve the LP relaxation of the original IP

2. Construct a bipartite graph based on the fractional solution

3. Determinsticly round the fractional solution to integral with the help of the bipartite graph 

In [157]:
def genAssign(p,c,T):
    x = getLPSol(p,c,T)
    B = buildGraph(x,p)
    res = detRound(B,n,m,n)
    return res

## LP Relaxation


Consider the following LP relaxation:

\begin{equation*}
\begin{array}{ll@{}ll}
\text{minimize}  & \displaystyle\sum\limits_{i,j} x_{ij}&c_{ij} &\\
\text{subject to}& \displaystyle\sum\limits_{j=1}^m   &x_{ij} = 1,  &i=1 ,..., n\\
                  & \displaystyle\sum\limits_{i=1}^n   &p_{ij}x_{ij} \leq  T,  &j=1 ,\cdots, m\\
                 &                                                &x_{ij} \geq 0, &i=1,\cdots,n,\\
                 & & &j=1 ,\cdots, m
\end{array}
\end{equation*}

Solving this LP gives us a fractional solution $\mathbf{x}\in [0,1]^{n\times m}$.

In [158]:
def getLPSol(p, c, T):
    """
    Get a solution to the LP. There are n jobs and m machines
    Inputs:
        p: an mxn matrix
        c: an mxn matrix
        T: a number
    Output: an mxn matrix
    """
    # get LP dimensions
    m = p.shape[0]
    n = p.shape[1]

    d = np.empty(m * n)         # coefficient vector
    A_ub = np.zeros((m, m * n)) # upper bound constraint matrix
    b_ub = T * np.ones(m)       # upper bound constraint vector
    A_eq = np.zeros((n, m * n)) # equality constraint matrix
    b_eq = np.ones(n)           # equality constraint vector
    bounds = []                 # bounds on variables

    # construct coefficient vector and set variable bounds
    for i in range(m):
        for j in range(n):
            # construct coefficient vector
            d[i * n + j] = c[i, j]
            # set variable bounds
            if p[i, j] > T:
                bounds.append((0, 0))
            else:
                bounds.append((0, None))

    # construct upper bound matrix
    for i in range(m):
        for j in range(n):
            A_ub[i, i * n + j] = p[i, j]

    # construct equality constraint matrix
    for j in range(n):
        for i in range(m):
            A_eq[j, i * n + j] = 1

    # get LP solution
    res = linprog(d, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    return np.reshape(res.x, (m, n))

## Constructing Bipartite Graph

Let $k_j = \lceil\sum_{i=1}^n x_{ij}\rceil$. Define a bipartite graph $(A,B)$, where $A=[n]$ and $B=\{(j,s)|j\in[m], s\in[s_j]\}$. We illustrate the edges in the bipartite graph by only looking at the case $j=1$.

For $j=1$, consider only the jobs $i$ such that $x_{i1}>0$ (assume this is $[n]$ wlog). Assume that $p_{11}\geq p_{21}\geq\cdots p_{n1}$ without loss of generality (sort them if necessary). For each $i'$, we track the partial sum $\sum_{i=1}^{i'}x_{i1}$.

- If $\sum_{i=1}^{i'}x_{i1}\leq s, \sum_{i=1}^{i'-1}x_{i1}\geq s-1$, then add an edge $(i,(1,s))$ with weight $x_{i'1}$;
- If $\sum_{i=1}^{i'}x_{i1}> s, \sum_{i=1}^{i'-1}x_{i1}< s-1$, then add an edge $(i,(1,s-1))$ with weight $s-\sum_{i=1}^{i'-1}x_{i1}$ and an edge $(i,(1,s))$ with weight $\sum_{i=1}^{i'}x_{i1}-s$.

In [154]:
def buildGraph(x, p):
    """
    Convert an LP solution to a bipartite graph with fractional
    edge weights where one side represents jobs and the other side
    represents slots
    Inputs:
        x: an mxn matrix
        p: an mxn matrix. These are the job completion times.
        It's required because we need to sort the rows
        of x based on the entries of p.
    Output: a nxmxn array B, where B[j, i, s] represents the weight
    on the edge from job j to slot s of machine i.
    """
    m = x.shape[0]
    n = x.shape[1]

    B = np.zeros((n, m, n))

    for i in range(m):
        s = 0       # current slot in machine i
        left = 1    # space left in current slot
        x_indices = np.argsort(-p[i]) # indices of x when p_i is in nondecreasing order
        for j in x_indices:
            y = x[i, j]
            if y <= left:
                # we can fit this job into slot s
                B[j, i, s] = y
                left -= y
            else:
                # we need to split this job into two slots
                B[j, i, s] = left       # fill up current slot
                s += 1                  # increment slot
                B[j, i, s] = y - left   # put remaining weight in new slot
                left = 1 - (y - left)   # update space left in new slot

    return B

## Deterministic Rounding

Note that the bipartite graph is a fractional min-cost perfect matching of the underlying cost function $C(i,(j,s))=c_{ij}$. It can be shown that all extreme points of the polytope are integral, and there is a way to convert the fractional solution into an optimal integral one within polynomial time.

Call all edges with weight in the range $(0,1)$ _unsaturated_ and $\{0,1\}$ _saturated_. We present an iterative method such that after one loop, the number of unsaturated edges decreases by at least $1$, and the sum of weights attached to a left vertex is preserved as $1$. Since at the end there is no unsaturated edge left and each left vertex is attached to edges with weights summing up to $1$, each left vertex is mapped to a right vertex via the unique edge with weight $1$. We will see shortly that this map is also injective, proving that it is a perfect matching on the left side.

Consider the following two cases:

1. There is a cycle of unsaturated edges. Color them by blue and red alternately. By increasing the weights of all red edges by a small amount $\delta$ and decreasing the weights of blue ones by $\delta$, no LP constraints are violated provided that $\delta$ is sufficiently small. As this works for both positive and negative $\delta$s, the original optimal solution lies in the middle of a line segment on the polytope, and every  point on the line is optimal as well since the objective function is linear. We can thus move $\delta$ towards the positive direction until one of the red edges comes to $1$ or one of the blue edges comes to $0$. Either way we decreased the number of unsaturated edges by at least $1$.

2. Call a vertex _single_ if it is only attached to an unsaturated edge, then the weight on the vertex is easily seen as equal to the the weight of the edge. Note that no left vertex can be single since they all have weight exactly $1$. Suppose now there is no cycle of unsaturated edge left, then each maximum path of unsaturated edge must start and end at single vertices. Choose a maximum path and color the edges by blue and red alternatively. Similar to case 1, perturbing the weight of all blue and red edges by a small amount $\delta$ does not violate any of the LP constraints, and we can freely move $\delta$ until one red edge reaches $1$ or one blue edge reaches $0$. Preprocessing could be handy if we merge all single nodes as one beforehand, as this case would fall into case 1 as well.

In [161]:
def detRound(X,n,m,k):
    '''
    Deterministicly round the input fractional min-cost perfect matching to a integral one and output it.
        Input:
            X: fractional perfect matching represented by adjacency matrix
            n,m,k: dimensions
        Output:
            res: result of the assignment
    '''
    res = np.ones(n)*(-1)
    # Preprocessing - get all edges w/ weight 1
    for i in range(n):
        for j in range(m):
            for s in range(k):
                if(X[i][j][s] == 1):
                    res[i] = j
                    X[i][j][s] = 0
    
    # Preprocessing - merge all single vertices as one
    B = np.reshape(X, (X.shape[0],m*k))
    B = np.hstack((B,np.zeros(n).reshape((-1,1))))
    single_dict = {}
    for j in range(m*k):
        if(np.count_nonzero(B[:,j]) == 1):
            nz = np.nonzero(B[:,j])[0]
            B[:,-1] += B[:,j]
            B[nz,j] = 0
            single_dict[nz] = j // k
    
    while(list(res).count(-1) != 0): # not all jobs are assigned
        #st = np.where(res==-1)[0] # starting point of a loop
        cyc = cycle_bipartite(B)  # finding a cycle  
        pairs = round_cycle(B, cyc, single_dict) # round the cycle
        if(len(pairs)!= 0): #  record all fulfilled edges
            for j in range(len(pairs[:,0])):
                if(pair[1] != B.shape[1]-1): # actual cycle
                    res[pairs[j][0]]= pairs[j][1] // k
                else:  # augmented path
                    res[pairs[j][0]] =  single_dict(pairs[j][0])
    return res

In [218]:
def cycle_bipartite(x):
    '''
    Find a cycle in a bipartite graph.
        Input: 
            x: a bipartite graph encoded in an (n,m) matrix
            st: candidate list of left starting point of a potential cycle
        Output:
            cyc: indices of a cycle, where there is an edge cyc[i][0]->cyc[i][1] and cyc[i][1]->cyc[i+1][0]
        
    '''
    #TODO: Correct this part
    cl = []
    cr = []
    st = range(len(x[0]))
    xcopy = np.copy(x)
    return dfs_left(xcopy,cl,cr,st)
    
def dfs_left(x,cl,cr, l):
    cl.append(-1)
    for i in range(len(l)):
        cl[-1]=l[i]
        s=-1
        t=-1
        if(len(cr)!= 0):            
            s = cl[-1]
            t = cr[-1]
            x[s,t]=0
        llist = np.where(x[cl[-1],:] != 0)[0]
        cyc = dfs_right(x, cl, cr, llist)
        if(cyc != None):
            return cyc   
        if(len(cr)!= 0):
            x[s,t]=1
    cl.pop()
    return None

def dfs_right(x,cl,cr, l):  
    cr.append(-1)
    for i in range(len(l)):
        cr[-1]=l[i]
        s = cl[-1]
        t = cr[-1]
        x[s,t]=0
        rlist = np.where(x[:,t] != 0)[0]      
        if(cl[0] in rlist):
            cyc = [cl,cr]
            return cyc
        cyc = dfs_left(x, cl, cr, rlist)
        if(cyc != None):
            return cyc       
        x[s,t]=1
    cr.pop()
    return None

In [150]:
def round_cycle(B, cyc, sd):
    '''
        Input:
            B: an adjacency matrix
            cyc: a cycle in B
        Output:
            pairs: lists of edges with weight 1
            B is rounded such that at least one edge in the cycle in the graph is saturated
    '''
    l = len(cyc[:,0])
    st = cyc[0][0]
    w = np.amin([1-B[cyc[i][0],cyc[i][1]] for i in range(l)])
    d = np.amin([B[cyc[i][0],cyc[i-1][1]] for i in range(1,l)])
    c = min([w,d,B[st][cyc[-1][1]]])
    
    pairs = []
    curl = st
    for i in range(l):

        # increase the weights of red edges
        curr = cyc[i][1]
        B[curl][curr] += c
        if(B[curl][curr] == 1.): # new integral edge found
            B[curl][curr] = 0
            pairs.append([curl,curr])
        # decrease the weights of blue edges
        if(i < l - 1):
            curl = cyc[i+1][0]
        else:
            curl = st
        B[curl][curr] -= c
        if(B[curl][curr] == 0):
            if(np.count_nonzero(B[:,curr]) == 1):
                B[:,curr] = 0
                sd[curl] = curr // k
    return pairs

# Test

## Sanity Test

## Correctness Test

## Performance Test

In [219]:
G = [[0,1,1,0],[1,1,0,0],[1,1,0,0],[0,0,1,1],[0,0,1,1]]
cycle_bipartite(G)

[[1, 2], [0, 1]]